# Quantitative Trading Strategies Final Project Draft
### Sean Lin (STUDENT ID) | Thomas McDonnell (STUDENT ID) | Ben Panovich (12365148) | Madison Rusch (12365298)

## Data Collection

In [83]:
import pandas as pd
import datetime as dt
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt 
import functools
import quandl
import statsmodels.regression.linear_model as linreg
import requests

@functools.lru_cache()
def fetch_quandl_table(table, begin_date = None, end_date = None, ticker=None):
    qdata = quandl.get_table(table,
                      date = { 'gte': begin_date, 'lte': end_date },
                      # qopts = {"columns":["date", "adj_close"]},
                      ticker = ticker,
                      api_key='bXgDCzJUfS5ZxasswLcN',
                      paginate=True)
    return qdata

@functools.lru_cache()
def fetch_quandl(asset, begin_date = None, end_date = None, api_key = 'bXgDCzJUfS5ZxasswLcN'):
    qdata = quandl.get(asset,
                      start_date = begin_date,
                      end_date = end_date,
                      api_key= api_key,
                      paginate=True)
    return qdata

@functools.lru_cache()
def fetch_options_price(option, date = '2014-01-01', end_date = '2022-12-31'):
    data_url = f'https://api.polygon.io/v1/open-close/{option}/{date}?adjusted=true&apiKey=cIrLrp5MiBJNGpjRm4hv7hoSbNVirkxx'
    # data_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?apiKey=cIrLrp5MiBJNGpjRm4hv7hoSbNVirkxx'
    # data_url = data_url + f'&expiration_data.gte={begin_date}'
    # data_url = data_url + f'&expiration_data.lte={end_date}'
    response = requests.get(data_url)
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code
    

In [3]:
# Get VIX close (this will be Y in our regression)

VIX_data = pd.read_csv('../Final Project Data/VIX_History.csv')
VIX_data['DATE'] = pd.to_datetime(VIX_data['DATE'])
VIX_data = VIX_data[VIX_data['DATE'] > '2013-12-31'].set_index('DATE').drop(columns=['OPEN', 'HIGH', 'LOW']).rename(columns={'CLOSE': 'VIX'})
VIX_data = VIX_data[VIX_data.index < '2023-01-01']
display(VIX_data)

,VIX
DATE,
2014-01-02,14.23
2014-01-03,13.76
2014-01-06,13.55
2014-01-07,12.92
2014-01-08,12.87
...,...
2022-12-23,20.87
2022-12-27,21.65
2022-12-28,22.14


In [4]:
# Grab (and filter) event data from FxStreet

event_data = pd.read_csv('../Final Project Data/Economic Events.csv')
event_data['DATE'] = pd.to_datetime(event_data['Start']).dt.date
event_data = event_data.set_index('DATE')

display(event_data.head())

,Id,Start,Name,Impact,Currency
DATE,,,,,
2014-01-03,5eb8c082-f21f-48cf-97ee-6deeef147743,01/03/2014 19:30:00,Former Fed's Chair Bernanke speech,HIGH,USD
2014-01-06,772b7026-8faf-4daa-a1d6-09f73e8b8bc8,01/06/2014 15:00:00,ISM Services PMI,HIGH,USD
2014-01-06,9baa22c0-ec0f-408a-83d6-f15402804ea5,01/06/2014 22:30:00,Fed Chairman Nomination Vote,HIGH,USD
2014-01-08,21996eee-50f9-4449-9e63-ca90b527e1d3,01/08/2014 19:00:00,FOMC Minutes,HIGH,USD
2014-01-10,0c37fc76-b9d5-46e2-9030-32f6d5e4dc51,01/10/2014 13:30:00,Nonfarm Payrolls,HIGH,USD


In [5]:
# Set event column to True if there's an event, False otherwise

idx = pd.date_range('2014-01-01', '2022-12-31')
VIX_data = VIX_data.reindex(idx, fill_value=None)
VIX_data['Event'] = np.where(VIX_data.index.isin(event_data.index.values), True, False)
VIX_data['Days until Event'] = np.where(VIX_data['Event'] == True, 0, VIX_data.groupby((VIX_data['Event'] == True).cumsum()).cumcount(ascending=False)+1)


# TODO: do we want weekends? remove this if so, and change fill_value above to .ffill()
VIX_data = VIX_data.dropna(subset=['VIX'])
display(VIX_data.head(15))

,VIX,Event,Days until Event
2014-01-02,14.23,False,1
2014-01-03,13.76,True,0
2014-01-06,13.55,True,0
2014-01-07,12.92,False,1
2014-01-08,12.87,True,0
2014-01-09,12.89,False,1
2014-01-10,12.14,True,0
2014-01-13,13.28,False,3
2014-01-14,12.28,False,2
2014-01-15,12.28,False,1


In [6]:
# Grab the VIX data and shift it for regression purposes

VIX_data['VIX Shifted'] = VIX_data['VIX'].shift(1)
display(VIX_data)

,VIX,Event,Days until Event,VIX Shifted
2014-01-02,14.23,False,1,NaN
2014-01-03,13.76,True,0,14.23
2014-01-06,13.55,True,0,13.76
2014-01-07,12.92,False,1,13.55
2014-01-08,12.87,True,0,12.92
...,...,...,...,...
2022-12-23,20.87,True,0,21.97
2022-12-27,21.65,False,5,20.87
2022-12-28,22.14,False,4,21.65
2022-12-29,21.44,False,3,22.14


In [7]:
# Consumer Sentiment (University of Michigan Consumer Survey, Index of Consumer Sentiment)
# https://data.nasdaq.com/data/UMICH/SOC1-university-of-michigan-consumer-surveyindex-of-consumer-sentiment
# NOTE: this is monthly data, so I forward filled
consumer_sentiment_data = fetch_quandl('UMICH/SOC1', begin_date='2013-12-01', end_date='2022-12-31')
VIX_data['Consumer Sentiment Index'] = consumer_sentiment_data
VIX_data['Consumer Sentiment Index'] = VIX_data['Consumer Sentiment Index'].fillna(method='ffill')
# Fill in the first month with Dec 2013 Index value
VIX_data['Consumer Sentiment Index'] = VIX_data['Consumer Sentiment Index'].fillna(value=consumer_sentiment_data.iloc[0].Index)
display(VIX_data)

,VIX,Event,Days until Event,VIX Shifted,Consumer Sentiment Index
2014-01-02,14.23,False,1,NaN,82.5
2014-01-03,13.76,True,0,14.23,82.5
2014-01-06,13.55,True,0,13.76,82.5
2014-01-07,12.92,False,1,13.55,82.5
2014-01-08,12.87,True,0,12.92,82.5
...,...,...,...,...,...
2022-12-23,20.87,True,0,21.97,56.8
2022-12-27,21.65,False,5,20.87,56.8
2022-12-28,22.14,False,4,21.65,56.8
2022-12-29,21.44,False,3,22.14,56.8


In [8]:
# Fetch Options chain for each expiry day
options = ['O:SPY221215P00400000', 'O:SPY221215P00350000', 'O:SPY221216P00400000', 'O:SPY221216P00350000']
dates = pd.date_range('2022-12-01', '2022-12-15')
dates = [date.strftime('%Y-%m-%d') for date in dates]
options_price_data = pd.DataFrame(index=pd.date_range('2014-01-01', '2022-12-31'), columns=options)
for option in options:
    for date in dates:
        result = fetch_options_price(option, date=date)
        if not isinstance(result, int):
            options_price_data.loc[date, option] = result['close']
display(options_price_data[options_price_data.index > '2022-12-01'])

,O:SPY221215P00400000,O:SPY221215P00350000,O:SPY221216P00400000,O:SPY221216P00350000
2022-12-02,3.48,0.1,4.37,0.12
2022-12-03,NaN,NaN,NaN,NaN
2022-12-04,NaN,NaN,NaN,NaN
2022-12-05,6.26,0.12,7.4,0.15
2022-12-06,9.48,0.2,10.86,0.22
2022-12-07,10.05,0.14,11.42,0.2
2022-12-08,7.98,0.1,9.12,0.14
2022-12-09,9.57,0.09,10.85,0.13
2022-12-10,NaN,NaN,NaN,NaN
2022-12-11,NaN,NaN,NaN,NaN


In [15]:
# IVW Options
options = ['O:IVW221215C00060000']#, 'O:IVW221215P00055000', 'O:IVW221216P00055000', 'O:IVW221216P00055000']
dates = pd.date_range('2022-12-01', '2022-12-15')
dates = [date.strftime('%Y-%m-%d') for date in dates]
options_price_data = pd.DataFrame(index=pd.date_range('2014-01-01', '2022-12-31'), columns=options)
for option in options:
    for date in dates:
        result = fetch_options_price(option, date=date)
        if not isinstance(result, int):
            options_price_data.loc[date, option] = result['close']
display(options_price_data[options_price_data.index > '2022-12-01'])


,O:IVW221215C00060000
2022-12-02,NaN
2022-12-03,NaN
2022-12-04,NaN
2022-12-05,NaN
2022-12-06,NaN
2022-12-07,NaN
2022-12-08,NaN
2022-12-09,NaN
2022-12-10,NaN
2022-12-11,NaN


In [10]:
# Get historical 10-day volatilities
'''
SPY: Tracks S&P 500
IVW: Tracks growth stocks in S&P 500
RSP: Equally weights S&P 500
IVV: iShares S&P 500
'''
ETFS = ['VOL/SPY', 'VOL/IVW', 'VOL/RSP', 'VOL/IVV']
for ticker in ETFS:
    historical_vol = fetch_quandl('VOL/SPY', begin_date='2014-01-01', end_date='2022-12-31', api_key='iJQ34VBCfaVGdxiuTpSv')
    ticker = ticker[4:]
    VIX_data[f'{ticker} Vol'] = historical_vol['Hv10']
display(VIX_data)

,VIX,Event,Days until Event,VIX Shifted,Consumer Sentiment Index,SPY Vol,IVW Vol,RSP Vol,IVV Vol
2014-01-02,14.23,False,1,NaN,82.5,0.0834,0.0834,0.0834,0.0834
2014-01-03,13.76,True,0,14.23,82.5,0.0780,0.0780,0.0780,0.0780
2014-01-06,13.55,True,0,13.76,82.5,0.0787,0.0787,0.0787,0.0787
2014-01-07,12.92,False,1,13.55,82.5,0.0820,0.0820,0.0820,0.0820
2014-01-08,12.87,True,0,12.92,82.5,0.0821,0.0821,0.0821,0.0821
...,...,...,...,...,...,...,...,...,...
2022-12-23,20.87,True,0,21.97,56.8,0.2211,0.2211,0.2211,0.2211
2022-12-27,21.65,False,5,20.87,56.8,0.1781,0.1781,0.1781,0.1781
2022-12-28,22.14,False,4,21.65,56.8,0.1668,0.1668,0.1668,0.1668
2022-12-29,21.44,False,3,22.14,56.8,0.1987,0.1987,0.1987,0.1987


In [79]:
# Get Commodities Futures Pricing Data and try to map correlation to VIX
securities = [
              'OWF/CMX_GC_GC', # Gold
              'OWF/CMX_SI_SI', # Silver
              'OWF/NYM_NG_NG', # Natural Gas
              'OWF/NYM_RB_RB', # Gasoline
              'OWF/CME_LC_LC', # Live Cattle
              'OWF/CBT_W_W',   # Wheat
              'OWF/NYM_CL_CL', # Crude Oil
              'OWF/CBT_C_C',   # Corn
              # 'OWF/ICE_CT_CT', # Cotton
              # 'OWF/CME_LH_LH'  # Lean Hogs
              ]
dates = [
         'H2014', 'M2014', 'U2014', 'Z2014',
         'H2015', 'M2015', 'U2015', 'Z2015',
         'H2016', 'M2016', 'U2016', 'Z2016',
         'H2017', 'M2017', 'U2017', 'Z2017',
         'H2018', 'M2018', 'U2018', 'Z2018',
         'H2019', 'M2019', 'U2019', 'Z2019',
         'H2020', # 'M2020', 'U2020', 'Z2020',
        #  'H2021', 'M2021', 'U2021', 'Z2021', 
        #  'H2022', 'M2022', 'U2022', 'Z2022'
         ]

prices = pd.DataFrame(index = pd.date_range('2014-01-01', '2019-12-31'))
for security in securities[0:]:
  finalized_data = pd.DataFrame()
  trim_start = "2014-01-01"
  trim_end = "2019-12-31"
  for month in dates:
    data = fetch_quandl(f'{security}_{month}_IVM', begin_date=trim_start, end_date=trim_end)
    # data = clean_quandl_columns(data)
    # data = data[data['DtT'] > 30]
    if finalized_data.empty:
      finalized_data = data
    else:
      finalized_data = finalized_data.append(data)
    trim_start = finalized_data.index[-1] + pd.DateOffset(1)
  prices[security] = finalized_data['Future']

display(prices)

C:\Users\madrs\AppData\Local\Temp\ipykernel_34600\1282585951.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalized_data = finalized_data.append(data)
C:\Users\madrs\AppData\Local\Temp\ipykernel_34600\1282585951.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalized_data = finalized_data.append(data)
C:\Users\madrs\AppData\Local\Temp\ipykernel_34600\1282585951.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalized_data = finalized_data.append(data)
C:\Users\madrs\AppData\Local\Temp\ipykernel_34600\1282585951.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalized_data = finalized_data.append(data)
C:\Users

,OWF/CMX_GC_GC,OWF/CMX_SI_SI,OWF/NYM_NG_NG,OWF/NYM_RB_RB,OWF/CME_LC_LC,OWF/CBT_W_W,OWF/NYM_CL_CL,OWF/CBT_C_C
2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02,1225.9,20.128,4.296,2.7087,NaN,597.000,95.62,420.625
2014-01-03,1239.4,20.211,4.282,2.6633,NaN,605.875,94.14,423.625
2014-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2019-12-27,1524.1,17.943,2.173,1.7545,NaN,556.250,61.53,390.000
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-30,1524.6,18.001,2.157,1.7358,NaN,556.000,61.44,388.250


In [81]:
prices=prices.dropna()
commodities_price_returns = prices.pct_change()
display(commodities_price_returns)

,OWF/CMX_GC_GC,OWF/CMX_SI_SI,OWF/NYM_NG_NG,OWF/NYM_RB_RB,OWF/CME_LC_LC,OWF/CBT_W_W,OWF/NYM_CL_CL,OWF/CBT_C_C
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,-0.006862,-0.015719,-0.003968,0.010376,-0.004199,-0.005364,0.002778,-0.004382
2014-01-08,-0.003333,-0.012533,-0.021561,-0.007144,0.004584,-0.022817,-0.013960,-0.021127
2014-01-09,0.003099,0.007370,-0.046228,-0.004909,-0.000730,-0.007854,-0.006917,-0.011990
2014-01-10,0.014310,0.027435,0.009794,0.009227,0.000731,-0.026102,0.011536,0.050667
...,...,...,...,...,...,...,...,...
2019-11-29,0.008254,0.002990,-0.077633,-0.048294,-0.001853,0.028470,-0.049611,0.021433
2019-12-02,-0.002368,-0.008184,0.020018,-0.007980,-0.003713,-0.012226,0.013823,0.001967
2019-12-03,0.010376,0.016621,0.028992,-0.005970,-0.001656,-0.018683,0.001435,-0.001963
2019-12-04,-0.002886,-0.019249,-0.008236,0.027058,-0.009125,0.004522,0.039771,-0.006885


In [143]:
correlations_to_vix = pd.DataFrame()
for column in commodities_price_returns.columns.values:
    # display(commodities_price_returns[column])
    vix_returns = VIX_data['VIX'].pct_change()
    correlations_to_vix[column] = (commodities_price_returns[column]).corr(vix_returns[abs(vix_returns) > 0.1])
    print(column)
    print(commodities_price_returns[column].corr(vix_returns[abs(vix_returns) > 0.07]))
    

OWF/CMX_GC_GC
0.19321584795902977
OWF/CMX_SI_SI
-0.034657114885321874
OWF/NYM_NG_NG
-0.05546152913314549
OWF/NYM_RB_RB
-0.2445179116873623
OWF/CME_LC_LC
-0.054186901336274934
OWF/CBT_W_W
-0.10076761456372502
OWF/NYM_CL_CL
-0.28699863747677223
OWF/CBT_C_C
-0.1223352600636498


In [120]:
currencies = ['CUR/GBP', 'CUR/AUD', 'CUR/CAD', 'CUR/CHF', 'CUR/CNY', 'CUR/EUR', 'CUR/HKD', 'CUR/INR', 'CUR/JPY', 'CUR/MXN']
for currency in currencies:
    returns = fetch_quandl(currency, begin_date='2014-01-01', end_date='2019-12-31')['RATE'].pct_change()
    # display(eurodollar_returns)
    vix_returns = VIX_data['VIX'].pct_change()
    print(currency)
    print(returns.corr(vix_returns[abs(vix_returns) > 0.07]))

CUR/GBP
0.136080884305824
CUR/AUD
0.17739909028690523
CUR/CAD
0.11999475264037972
CUR/CHF
-0.10372436372532422
CUR/CNY
0.04495214382298298
CUR/EUR
0.017284414069671995
CUR/HKD
0.09660414480123619
CUR/INR
0.2721254900802231
CUR/JPY
-0.26181610645310643
CUR/MXN
0.2807773269400704


In [152]:
bond_data = fetch_quandl('YC/USA10Y', begin_date='2014-01-02', end_date='2019-12-31')
bond_data['Returns'] = bond_data['Rate'].pct_change()
bond_data['VIX Returns'] = VIX_data['VIX'].pct_change()

# bond_data = bond_data[abs(bond_data['VIX Returns']) > 0.07]
display(bond_data.corr())
# display(VIX_data)
VIX_data['VIX Returns'] = VIX_data['VIX'].pct_change()
big_vix = VIX_data[VIX_data['VIX Returns'] > 0.07]
display(big_vix[big_vix.index > '2020-01-01'])


# print(bond_data['VIX Returns'].quantile(0.85))
# bond_data['VIX Returns'].hist(bins=30)

# print(bond_data['Returns'].corr(bond_data['VIX Returns']))

,Rate,Returns,VIX Returns
Rate,1.000000,0.040803,0.001365
Returns,0.040803,1.000000,-0.302103
VIX Returns,0.001365,-0.302103,1.000000


,VIX,Event,Days until Event,VIX Shifted,Consumer Sentiment Index,SPY Vol,IVW Vol,RSP Vol,IVV Vol,VIX Returns
2020-01-03,14.02,True,0,12.47,99.3,0.0933,0.0933,0.0933,0.0933,0.124298
2020-01-24,14.56,False,4,12.98,99.3,0.0837,0.0837,0.0837,0.0837,0.121726
2020-01-27,18.23,False,1,14.56,99.3,0.1286,0.1286,0.1286,0.1286,0.252060
2020-01-31,18.84,False,3,15.49,99.8,0.1680,0.1680,0.1680,0.1680,0.216269
2020-02-18,14.83,False,1,13.68,99.8,0.0736,0.0736,0.0736,0.0736,0.084064
...,...,...,...,...,...,...,...,...,...,...
2022-11-28,22.21,False,2,20.50,59.9,0.1478,0.1478,0.1478,0.1478,0.083415
2022-12-05,20.75,True,0,19.06,56.8,0.2550,0.2550,0.2550,0.2550,0.088667
2022-12-12,25.00,False,1,22.83,56.8,0.1838,0.1838,0.1838,0.1838,0.095050
2022-12-15,22.83,False,7,21.14,56.8,0.2070,0.2070,0.2070,0.2070,0.079943


## Tradeable Assets

### Commodity Futures

We analyzed multiple commodity futures from the OWF data set, and found that Gold was the most highly correlated to the VIX returns of over 7%.

When predicted VIX returns are greater than 0.05 (closely tied to actual movements > 0.07), we will:
- Long the SPY ETF call option (with delta ~0.5)
    - Buy at the money calls expiring in approximately 20 days
- Short SPY ETF to delta hedge the above
- Long Gold Futures
- Short the 10Y Bonds
- Short the USDJPY (ie buy Yen)

Exit postions in 2 trading days

Risk:
- Interest Rate movements by the FED (Bonds)
- Foreign Exchange Rate (JPY)
- Market Risk is prevented by delta hedge
- No inflation risk because of short time window

Funding Model:
- Self-Financing
    - SPY ETF funds the Call
    - Bonds/USDJPY fund the Gold Futures

Transaction Fees:
- Call Option: dollar premium
- SPY ETF: assume negligible
- Futures: negligible because of short trading window/no rollover
- Bonds: We need to buy the bonds with capital, pay the bank the risk free rate (for two days), earn the haircut while repoing the bonds, sell the bonds after two days to close position with capital
- USDJPY: Pay the rate differential of USD - JPY 

Steps:                                                                              CASH
1. Assume we have $1 million capital                                                $ 1,000,000
2. Use it as collateral to borrow $10 million dollars from the bank (Pay SOFR)      $10,000,000
3. Buy $10 million of US 10 year treasury bonds                                     $         0 
4. Reverse repo the $10 million Treasuries (Earn haircut)                           $10,000,000
5. Buy $5 million of ATM SPY ETF Call Options                                       $ 5,000,000
6. Short SPY ETF to be as close to delta neutral on the negative side as possible   $ 5,000,000 (capital here is earned when position is closed)
7. Buy $2 million JPY                                                               $ 3,000,000
8. Buy $2 million Gold Futures                                                      $ 1,000,000 (just in case we lose money, this lets us put on the next trade)

Closing the trade:
8. Sell the Gold Futures (based on pricing data)
7. Sell JPY (based on spot rate)
6. Buy the SPY ETF (based on prices)
5. Sell our call options
4. Selling the Treasuries
3. Pay off the $10 million bank loan
2. Amount remaining is profit

In [130]:
display(VIX_data[VIX_data['Returns'].pct_change() > 0.07])

KeyError: 'Returns'